# Imports and version check

In [1]:
from ipycanvas import Canvas
import numpy as np
import sys
import math

if not (sys.version_info.major >= 3 and sys.version_info.minor >= 7):
    print("dictionaries might not be ordered")
    sys.exit(1)


# Global constants for backtracking

In [2]:
# For backtracking

moves = {"R": [0, 2], "BR": [1, 1], "BL": [1, -1], "L": [0, -2], "TL": [-1, -1], "TR": [-1, 1]}

move_to_number = {i: j for i, j in zip([i for i in moves], range(len(moves)))}

number_to_move = {j: i for i, j in zip([i for i in moves], range(len(moves)))}

# 1: BLUE, 2: PINK, 3: VIOLET, 4: RED, 5: ORANGE, 6: YELLOW, 7: GREEN
# The rules for how these puzzles interact with the board follow the rules of the puzzle
pieces = {2: [0, 1, 1], 7: [0, 0, -1], 1: [0, 0, 2], 3: [0, 0], 4: [0, 1, 2], 5: [0, 1], 6: [0, 0, -2]}

def fill_piece(board, coords, direction, piece_colour):
    new_board = np.copy(board)
    new_board[tuple(coords)] = piece_colour
    current_coords = coords
    current_direction = direction
    for direction_change in pieces[piece_colour]:
        current_direction = number_to_move[(move_to_number[current_direction] + direction_change) % len(moves)]
        current_coords = current_coords + moves[current_direction]
        new_board[tuple(current_coords)] = piece_colour
    return new_board

# Global constants for visualisation

In [3]:
background_colour = "#23395d"
empty_stroke = "#000133"

orange = "#ff7400"
green = "#006400"
blue = "#0018f9"
red = "#bf0a30"
beige = "#eed9c4"
yellow = "#fee12b"
pink = "#fc0fc0"
purple = "#551054"

# 1: BLUE, 2: PINK, 3: PURPLE, 4: RED, 5: ORANGE, 6: YELLOW, 7: GREEN
code_to_colour = {1: (blue, beige), 2: (pink, beige), 3: (purple, beige), 4: (red, beige), 5: (orange, beige), 6: (yellow, beige), 7: (green, beige), 0: (background_colour, empty_stroke)}

We are going to create spatial coordinates for all the stars in the puzzle:

First row will have 7 stars, the second 6, the third 7 and the fourth 6 again.

Each star will be represented as a hexagon, and the hexagons that are direct neighbours will sit next to each other.

In [4]:
x_init = 30
x_step = 50

y_init = 30
y_step = 25 * math.sqrt(3)

radius = 20


number_horizontally = 7

visual_coords = []

for j in range(2):
    next_line = []
    visual_coords.append(next_line)
    for i in range(number_horizontally):
        x_coord = x_init + x_step * i
        y_coord = y_init + y_step * 2 * j
        next_line.append([x_coord, y_coord])

    next_line = []
    visual_coords.append(next_line)
    for i in range(number_horizontally - 1):
        x_coord = x_init + x_step/2 + x_step * i
        y_coord = y_init + y_step * (1 + 2 * j)
        next_line.append([x_coord, y_coord])

# Computational representation of an example state with a valid solution of the puzzle

In [5]:
board_size = (4, 13)

example_board = np.zeros(board_size)

example_board = fill_piece(example_board, np.array([0, 0]), "BR", 5)
example_board = fill_piece(example_board, np.array([0, 2]), "BR", 7)
example_board = fill_piece(example_board, np.array([0, 4]), "R", 4)
example_board = fill_piece(example_board, np.array([0, 8]), "R", 3)
example_board = fill_piece(example_board, np.array([2, 8]), "TR", 2)
example_board = fill_piece(example_board, np.array([3, 5]), "L", 1)
example_board = fill_piece(example_board, np.array([3, 7]), "R", 6)

print(example_board)

[[5. 0. 7. 0. 4. 0. 4. 0. 3. 0. 3. 0. 3.]
 [0. 5. 0. 7. 0. 4. 0. 4. 0. 2. 0. 2. 0.]
 [5. 0. 1. 0. 7. 0. 7. 0. 2. 0. 6. 0. 2.]
 [0. 1. 0. 1. 0. 1. 0. 6. 0. 6. 0. 6. 0.]]


In [6]:
print(example_board[(3, 11)])

6.0


# Function to visualise an example board

In [7]:
def draw_star(canvas, x, y, radius, fill_colour, stroke_colour):
    n_points = 6
    angles = (2 * math.pi / n_points) * np.arange(n_points)

    v_x = x + np.cos(angles) * radius
    v_y = y + np.sin(angles) * radius

    points = np.stack((v_x, v_y), axis=1)
    canvas.stroke_style = stroke_colour
    canvas.line_width = 3
    canvas.fill_style = fill_colour
    canvas.fill_polygon(points)
    canvas.stroke_polygon(points)

def visualise_board(board):
    canvas = Canvas(width = (number_horizontally - 1) * x_step + 2 * x_init, height = 3 * y_step + 2 * y_init)
    
    canvas.fill_style = background_colour
    canvas.fill_rect(0, 0, canvas.width, canvas.height)
    
    for i in range(board_size[0]):
        for j in range(board_size[1]):
            element = board[i, j]
            if i % 2 == 0 and j % 2 == 0:
                coords = visual_coords[i][j // 2]
                draw_star(canvas, coords[0], coords[1], radius, code_to_colour[element][0], code_to_colour[element][1])
            if i % 2 == 1 and j % 2 == 1:
                coords = visual_coords[i][(j - 1) // 2]
                draw_star(canvas, coords[0], coords[1], radius, code_to_colour[element][0], code_to_colour[element][1])
    
    return canvas

In [8]:
visualise_board(example_board)

Canvas(height=189, width=360)

# A version of function to fill a piece that will work with backtracking

In [9]:
def fill_coords_report_success(board, coords, piece_colour):
    y_coord = coords[0]
    x_coord = coords[1]
    
    if ((y_coord % 2) == 0) != ((x_coord % 2) == 0):
        print("error in board representation, impossible state")
        return False
    if y_coord < 0:
        return False
    if y_coord >= board_size[0]:
        return False
    if x_coord < 0:
        return False
    if x_coord >= board_size[1]:
        return False
    
    current_entry = board[tuple(coords)]
    if current_entry != 0:
        return False
    board[tuple(coords)] = piece_colour
    return True

def fill_piece_report_success(board, coords, direction, piece_colour):
    new_board = np.copy(board)
    result = fill_coords_report_success(new_board, coords, piece_colour)
    if not result:
        return False, board
    current_coords = coords
    current_direction = direction
    for direction_change in pieces[piece_colour]:
        current_direction = number_to_move[(move_to_number[current_direction] + direction_change) % len(moves)]
        current_coords = current_coords + moves[current_direction]
        result = fill_coords_report_success(new_board, current_coords, piece_colour)
        if not result:
            return False, board
    return True, new_board

In [10]:
example_board = np.zeros(board_size)

success, example_board = fill_piece_report_success(example_board, np.array([1, 5]), "R", 7)
print(success)
success, example_board = fill_piece_report_success(example_board, np.array([2, 8]), "BL", 2)
print(success)

True
True


In [11]:
example_board

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 7., 0., 0.],
       [0., 0., 0., 0., 0., 7., 0., 7., 0., 7., 0., 0., 0.],
       [0., 0., 0., 0., 2., 0., 0., 0., 2., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 2., 0., 2., 0., 0., 0., 0., 0.]])

In [12]:
example_board = np.zeros(board_size)

success, example_board = fill_piece_report_success(example_board, np.array([2, 2]), "R", 1)
print(success)
visualise_board(example_board)

True


Canvas(height=189, width=360)

# Recursive backtracking

In [13]:
def solve_puzzle(board, pieces_left):
    if pieces_left:
        current_piece = pieces_left[0]
        other_pieces = pieces_left[1:]
    
        for move in moves:
            for i in range(board_size[0]):
                for j in range(board_size[1]):
                    element = board[i, j]
                    if element == 0:
                        if i % 2 == 0 and j % 2 == 0 or i % 2 == 1 and j % 2 == 1:
                            piece_success, new_board = fill_piece_report_success(board, np.array([i, j]), move, current_piece)
                            if piece_success:
                                solution_success, solved_board = solve_puzzle(new_board, other_pieces)
                                if solution_success:
                                    return True, solved_board
        return False, board
    else:
        return True, board

In [17]:
board = np.zeros(board_size)
success, board = fill_piece_report_success(board, np.array([2, 6]), "TR", 5)
pieces_left = [i for i in pieces]
pieces_left.remove(5)
print(pieces_left)
canvas_challenge = visualise_board(board)
success, solution = solve_puzzle(board, pieces_left)
canvas_solution = visualise_board(solution)

[2, 7, 1, 3, 4, 6]


In [18]:
canvas_challenge

Canvas(height=189, width=360)

In [19]:
canvas_solution

Canvas(height=189, width=360)